# lgbm
transfer learning  
리더보드 점수 8 

In [0]:
import numpy as np 
import pandas as pd
from datetime import time, datetime

import os
from pathlib import Path


In [0]:
# 재생산성을 위해 시드 고정
np.random.seed(7)

http://machinelearningkorea.com/2019/09/29/lightgbm-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0/

# 1. 데이터 불러오기

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
folder = "Colab Notebooks/dacon"
project_dir = "temperature_forecasting"
data_folder = "_data/data_dacon"

base_path = Path("/content/drive/My Drive/")
project_path = base_path / folder / project_dir
data_path = base_path / data_folder / project_dir
os.chdir(project_path)

print(f"현재 디렉토리 위치: {os.getcwd()}")

현재 디렉토리 위치: /content/drive/My Drive/Colab Notebooks/dacon/temperature_forecasting


In [0]:
df = pd.read_csv(os.path.join(data_path,'train.csv'), index_col = 'id')
df.shape

(4752, 59)

In [0]:
df_test = pd.read_csv(os.path.join(data_path,'test.csv'), index_col = 'id')
df_test.shape

(11520, 40)

#### 시간 정보 넣어주기

In [0]:
min_in_day = 24*6
hour_in_day = 24

def time_data(frame):
    minute = (frame.index%144).astype(int)
    hour= pd.Series((frame.index%144/6).astype(int))

    minute_sin = np.sin(np.pi*minute/min_in_day) 
    minute_cos = np.cos(np.pi*minute/min_in_day)

    hour_sin  = np.sin(np.pi*hour/hour_in_day)
    hour_cos  = np.cos(np.pi*hour/hour_in_day)

    return minute_sin, minute_cos, hour_sin, hour_cos

In [0]:
# train data
print(df.shape)
df['time_m_sin'], df['time_m_cos'], df['time_h_sin'], df['time_h_cos'] = time_data(df)
print(df.shape)

(4752, 59)
(4752, 63)


In [0]:
# test data
print(df_test.shape)
df_test['time_m_sin'], df_test['time_m_cos'], df_test['time_h_sin'], df_test['time_h_cos'] = time_data(df_test)
print(df_test.shape)

(11520, 40)
(11520, 44)


In [0]:
n = 4320

|-|30일(train)|3일(train)|80일(test)|
|---|---|---|---|
|Y18|NULL|432개|NULL|
|Y00-Y17|4320개|NULL|NULL|
|X00-X17|4320개|432개|11520개|

In [0]:
df_30 = df[:n]
df_3 = df[n:n+n//10]

## time data

In [0]:
diff = np.mean(df[feat_temp][:n], axis=1)- np.mean(df[['Y15', 'Y16', 'Y09']][:n], axis=1)
diff

0       2.620000
1       2.753333
2       2.760000
3       2.786667
4       2.666667
          ...   
4315    0.120000
4316    0.026667
4317    0.273333
4318    0.253333
4319    0.260000
Length: 4320, dtype: float64

In [0]:
diff_144 =[]
for i in range(144):
    diff_144.append(diff[i: :144].mean())
diff_144 = np.array(diff_144)

In [0]:
normed_train_data['time'] = diff_144[normed_train_data.index%144]
normed_test_data['time'] = diff_144[normed_test_data.index%144]
normed_test_80['time'] = diff_144[normed_test_80.index%144]
normed_test_3['time'] = diff_144[normed_test_3.index%144]

# x

* feat_temp: 기온 
* feat_air_press: 현지기압
* feat_wind_speed: 풍속
* feat_rainfall: 일일 누적강수량 
* feat_sea_press: 해면기압
* feat_insol: 일일 누적일사량
* feat_humid: 습도
* feat_dir_wind: 풍향



In [0]:
feat_time = ['time_m_sin', 'time_m_cos', 'time_h_sin', 'time_h_cos']
feat_temp = ['X00', 'X07', 'X28', 'X31', 'X32']
feat_air_press = ['X01', 'X06', 'X22', 'X27', 'X29']
feat_wind_speed = ['X02', 'X03', 'X18', 'X24', 'X26']
feat_rainfall = ['X04', 'X10', 'X21', 'X36', 'X39']
feat_sea_press = ['X05', 'X08', 'X09', 'X23', 'X33']
feat_insol = ['X11', 'X14', 'X16', 'X19', 'X34']
feat_humid = ['X12', 'X20', 'X30', 'X37', 'X38']
feat_dir_wind = ['X13', 'X15', 'X17', 'X25', 'X35']

feat_x = ['X00', 'X01', 'X02', 'X03', 'X04', 'X05', 'X06', 'X07', 'X08', 'X09', 
          'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16', 'X17', 'X18', 'X19', 
          'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 
          'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X36', 'X37', 'X38', 'X39']

feat_x_t = feat_x + feat_time 

### 일일누적일사량 
데이터가 의미 없는 열은 삭제 

In [0]:
remove_col =  ['X14', 'X16', 'X19']

In [0]:

for r in remove_col:
    feat_insol.remove(r)
    feat_x.remove(r)

In [0]:
df = df.drop(remove_col, axis = 1)

In [0]:
df_test = df_test.drop(remove_col, axis = 1)

## y

In [0]:
feat_y = ['Y00', 'Y01', 'Y02', 'Y03', 'Y04', 'Y05', 'Y06', 'Y07', 'Y08', 
          'Y09', 'Y10', 'Y11', 'Y12', 'Y13', 'Y14', 'Y15', 'Y16', 'Y17']

# model

lgbm model - transfer learning (x로 yi평균 맞추게 하고 x로 y18 fine tuning)  
  
boosting_type 3가지 해봄. rf 는 계속 learning 할 수 없어서 못해봄 

In [0]:
import lightgbm as lgb
import collections
from sklearn.metrics import mean_squared_error

In [0]:
## loss function
def mse1(y_pred, dataset):
    y_true = dataset.get_label()

    diff = abs(y_true - y_pred)
    less_then_one = np.array([0 if x < 1 else 1 for x in diff])

    y_pred = less_then_one * y_pred
    y_true = less_then_one *y_true
    
    score= mean_squared_error(y_true, y_pred)
    
    return 'score', score, False

In [0]:
## data set 
## x_col, y_col 정하기 
x_col = feat_temp + feat_air_press+ feat_rainfall+ feat_insol+feat_dir_wind+feat_time
y_col = ['Y15', 'Y16', 'Y09'] 

data = df_30[x_col]
label = df_30[y_col].mean(axis=1)
m1_train_data = lgb.Dataset(data[:n*9//10], label=label[:n*9//10], free_raw_data=False)
m1_validation_data = lgb.Dataset(data[n*9//10:], label=label[n*9//10:], free_raw_data=False)

In [0]:
## data set (finetune)
y_col = ['Y18'] 
m = 432
data = df_3[x_col]
label = df_3[y_col]
m2_train_data = lgb.Dataset(data[:m*9//10], label=label[:m*9//10], free_raw_data=False)
m2_validation_data = lgb.Dataset(data[m*9//10:], label=label[m*9//10:], free_raw_data=False)

## model - lgbm 1 성능 8 

In [0]:
## model parameter
params ={'boosting_type': 'gbdt',
 'learning_rate': 0.01,
 'num_leaves': 51,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            num_boost_round = 1000,
            verbose_eval=20,
            early_stopping_rounds=5,
            keep_training_booster=True) 

Training until validation scores don't improve for 5 rounds.
[20]	valid_0's l2: 25.9261	valid_0's score: 25.8947
[40]	valid_0's l2: 17.89	valid_0's score: 17.8423
[60]	valid_0's l2: 12.5096	valid_0's score: 12.4462
[80]	valid_0's l2: 8.85119	valid_0's score: 8.78657
[100]	valid_0's l2: 6.33741	valid_0's score: 6.26589
[120]	valid_0's l2: 4.62108	valid_0's score: 4.52814
[140]	valid_0's l2: 3.45482	valid_0's score: 3.3432
[160]	valid_0's l2: 2.73213	valid_0's score: 2.57824
[180]	valid_0's l2: 2.24944	valid_0's score: 2.07695
[200]	valid_0's l2: 1.92123	valid_0's score: 1.76038
[220]	valid_0's l2: 1.70745	valid_0's score: 1.53651
[240]	valid_0's l2: 1.56013	valid_0's score: 1.38247
[260]	valid_0's l2: 1.4708	valid_0's score: 1.28258
[280]	valid_0's l2: 1.41475	valid_0's score: 1.22998
[300]	valid_0's l2: 1.37734	valid_0's score: 1.17809
[320]	valid_0's l2: 1.35575	valid_0's score: 1.15451
[340]	valid_0's l2: 1.33874	valid_0's score: 1.1399
[360]	valid_0's l2: 1.32603	valid_0's score: 1.

In [0]:
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            num_boost_round = 1000,
            verbose_eval = 5,
            early_stopping_rounds = 10,
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


Training until validation scores don't improve for 10 rounds.
[375]	valid_0's l2: 2.86532	valid_0's score: 2.79594
[380]	valid_0's l2: 2.80834	valid_0's score: 2.73498
[385]	valid_0's l2: 2.7384	valid_0's score: 2.66527
[390]	valid_0's l2: 2.6784	valid_0's score: 2.58226
[395]	valid_0's l2: 2.6198	valid_0's score: 2.52723
[400]	valid_0's l2: 2.56536	valid_0's score: 2.43187
[405]	valid_0's l2: 2.48944	valid_0's score: 2.36265
[410]	valid_0's l2: 2.46199	valid_0's score: 2.33847
[415]	valid_0's l2: 2.40166	valid_0's score: 2.26203
[420]	valid_0's l2: 2.32853	valid_0's score: 2.19527
[425]	valid_0's l2: 2.2767	valid_0's score: 2.12611
[430]	valid_0's l2: 2.22105	valid_0's score: 2.07595
[435]	valid_0's l2: 2.18134	valid_0's score: 2.01728
[440]	valid_0's l2: 2.14015	valid_0's score: 1.98116
[445]	valid_0's l2: 2.09714	valid_0's score: 1.94225
[450]	valid_0's l2: 2.06206	valid_0's score: 1.91154
[455]	valid_0's l2: 2.0291	valid_0's score: 1.86008
[460]	valid_0's l2: 2.00543	valid_0's scor

In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

## model - lgbm dart 성능 8  
하이퍼파라메터 조정  
boosting_type: dart  
'num_leaves': 200  

In [0]:
## model parameter
params ={'boosting_type': 'dart',
 'learning_rate': 0.01,
 'num_leaves': 200,
 'random_state': 7,
 'num_boost_round': 10000,
 'objective': 'regression'}

In [0]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
         #   early_stopping_rounds=5,
            keep_training_booster=True) 

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	valid_0's l2: 99.3202	valid_0's score: 99.3202
[200]	valid_0's l2: 78.8948	valid_0's score: 78.8948
[300]	valid_0's l2: 57.8919	valid_0's score: 57.8919
[400]	valid_0's l2: 59.3466	valid_0's score: 59.3466
[500]	valid_0's l2: 33.8652	valid_0's score: 33.8652
[600]	valid_0's l2: 31.2215	valid_0's score: 31.2215
[700]	valid_0's l2: 25.4981	valid_0's score: 25.4966
[800]	valid_0's l2: 18.7528	valid_0's score: 18.7485
[900]	valid_0's l2: 12.9713	valid_0's score: 12.9602
[1000]	valid_0's l2: 9.91964	valid_0's score: 9.89259
[1100]	valid_0's l2: 10.3864	valid_0's score: 10.3692
[1200]	valid_0's l2: 14.605	valid_0's score: 14.5983
[1300]	valid_0's l2: 10.5363	valid_0's score: 10.5199
[1400]	valid_0's l2: 6.80191	valid_0's score: 6.73555
[1500]	valid_0's l2: 5.89446	valid_0's score: 5.81089
[1600]	valid_0's l2: 5.11424	valid_0's score: 5.01913
[1700]	valid_0's l2: 4.22429	valid_0's score: 4.13095
[1800]	valid_0's l2: 4.07641	valid_0's score: 3.98035
[1900]	valid_0's l2: 4.32499	valid_0's

In [0]:
## model parameter
params ={'boosting_type': 'dart',
 'learning_rate': 0.01,
 'num_leaves': 200,
 'random_state': 7,
 'objective': 'regression'}
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            num_boost_round = 11070,
            verbose_eval = 100,
          #  early_stopping_rounds = 10,
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


[10100]	valid_0's l2: 2.07584	valid_0's score: 1.92079
[10200]	valid_0's l2: 1.63488	valid_0's score: 1.40966
[10300]	valid_0's l2: 1.43416	valid_0's score: 1.22389
[10400]	valid_0's l2: 1.29232	valid_0's score: 1.05839
[10500]	valid_0's l2: 1.27152	valid_0's score: 0.999979
[10600]	valid_0's l2: 1.28671	valid_0's score: 1.02349
[10700]	valid_0's l2: 1.27666	valid_0's score: 1.0629
[10800]	valid_0's l2: 1.26651	valid_0's score: 1.05057
[10900]	valid_0's l2: 1.25879	valid_0's score: 1.02143
[11000]	valid_0's l2: 1.27332	valid_0's score: 1.0541
[11100]	valid_0's l2: 1.28798	valid_0's score: 1.06582
[11200]	valid_0's l2: 1.29541	valid_0's score: 1.07352
[11300]	valid_0's l2: 1.29779	valid_0's score: 1.0764
[11400]	valid_0's l2: 1.29674	valid_0's score: 1.12161
[11500]	valid_0's l2: 1.29959	valid_0's score: 1.10258
[11600]	valid_0's l2: 1.29981	valid_0's score: 1.10451
[11700]	valid_0's l2: 1.30843	valid_0's score: 1.11222
[11800]	valid_0's l2: 1.3164	valid_0's score: 1.11909
[11900]	valid

In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

## model - lgbm goss 성능 6.3
하이퍼파라메터 조정  
boosting_type: goss  
'num_leaves': 50  

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.005,
 'num_leaves': 50,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=50,
            num_boost_round = 5000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[50]	valid_0's l2: 23.6134	valid_0's score: 23.5687
[100]	valid_0's l2: 14.9521	valid_0's score: 14.9032
[150]	valid_0's l2: 9.69666	valid_0's score: 9.63253
[200]	valid_0's l2: 6.35474	valid_0's score: 6.28138
[250]	valid_0's l2: 2.51421	valid_0's score: 2.34909
[300]	valid_0's l2: 1.79569	valid_0's score: 1.6187
[350]	valid_0's l2: 1.56637	valid_0's score: 1.38484
[400]	valid_0's l2: 1.47389	valid_0's score: 1.28815
[450]	valid_0's l2: 1.42382	valid_0's score: 1.24602
[500]	valid_0's l2: 1.39097	valid_0's score: 1.21737
[550]	valid_0's l2: 1.40468	valid_0's score: 1.22529
Early stopping, best iteration is:
[553]	valid_0's l2: 1.27701	valid_0's score: 1.0829


In [0]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 20,
            early_stopping_rounds = 20,
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 20 rounds.
[600]	valid_0's l2: 2.82579	valid_0's score: 2.75336
[620]	valid_0's l2: 2.69571	valid_0's score: 2.62233
[640]	valid_0's l2: 2.58148	valid_0's score: 2.46921
[660]	valid_0's l2: 2.46662	valid_0's score: 2.34215
[680]	valid_0's l2: 2.39343	valid_0's score: 2.2566
Did not meet early stopping. Best iteration is:
[683]	valid_0's l2: 2.37456	valid_0's score: 2.23866


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

## model - lgbm goss - 2 성능 5.89
하이퍼파라메터 조정  
boosting_type: goss  
'num_leaves': 100  

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.0005,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
model1 = lgb.train(params, 
            m1_train_data,
            feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
            num_boost_round = 5000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[100]	valid_0's l2: 34.5363	valid_0's score: 34.4997
[200]	valid_0's l2: 31.4328	valid_0's score: 31.3968
[300]	valid_0's l2: 28.6231	valid_0's score: 28.585
[400]	valid_0's l2: 26.0767	valid_0's score: 26.0382
[500]	valid_0's l2: 23.737	valid_0's score: 23.698
[600]	valid_0's l2: 21.594	valid_0's score: 21.5529
[700]	valid_0's l2: 19.664	valid_0's score: 19.6162
[800]	valid_0's l2: 17.9271	valid_0's score: 17.8815
[900]	valid_0's l2: 16.3751	valid_0's score: 16.3297
[1000]	valid_0's l2: 14.9883	valid_0's score: 14.9313
[1100]	valid_0's l2: 13.7286	valid_0's score: 13.6747
[1200]	valid_0's l2: 12.5806	valid_0's score: 12.5319
[1300]	valid_0's l2: 11.5445	valid_0's score: 11.4957
[1400]	valid_0's l2: 10.6019	valid_0's score: 10.5508
[1500]	valid_0's l2: 9.72987	valid_0's score: 9.68106
[1600]	valid_0's l2: 8.94257	valid_0's score: 8.8913
[1700]	valid_0's l2: 8.23099	valid_0's score: 8.17841
[1800]	valid_0's l2: 7.59486	valid_

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.0001,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 20,
            early_stopping_rounds = 50,
            num_boost_round = 10000, 
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 50 rounds.
[4700]	valid_0's l2: 4.10741	valid_0's score: 4.06754
[4720]	valid_0's l2: 25.7795	valid_0's score: 25.7795
[4740]	valid_0's l2: 67.2597	valid_0's score: 67.2597
Early stopping, best iteration is:
[4699]	valid_0's l2: 3.54301	valid_0's score: 3.46206


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

In [0]:
model2.num_feature()

26

## model - lgbm goss - 3 (0315_1013) 성능 
하이퍼파라메터 조정  
metric: l2 (rmse랑 같음) 
boosting_type: goss  
'num_leaves': 100


In [0]:
## model parameter
params ={'metric': 'l2',
 'boosting_type': 'goss',
 'learning_rate': 0.0005,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
model1 = lgb.train(params, 
            m1_train_data,
           # feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
            num_boost_round = 10000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[100]	valid_0's l2: 34.5363
[200]	valid_0's l2: 31.4328
[300]	valid_0's l2: 28.6231
[400]	valid_0's l2: 26.0767
[500]	valid_0's l2: 23.737
[600]	valid_0's l2: 21.594
[700]	valid_0's l2: 19.664
[800]	valid_0's l2: 17.9271
[900]	valid_0's l2: 16.3751
[1000]	valid_0's l2: 14.9883
[1100]	valid_0's l2: 13.7286
[1200]	valid_0's l2: 12.5806
[1300]	valid_0's l2: 11.5445
[1400]	valid_0's l2: 10.6019
[1500]	valid_0's l2: 9.72987
[1600]	valid_0's l2: 8.94257
[1700]	valid_0's l2: 8.23099
[1800]	valid_0's l2: 7.59486
[1900]	valid_0's l2: 7.01422
[2000]	valid_0's l2: 6.50023
[2100]	valid_0's l2: 4.9885
[2200]	valid_0's l2: 4.06249
[2300]	valid_0's l2: 3.45814
[2400]	valid_0's l2: 3.04254
[2500]	valid_0's l2: 2.74099
[2600]	valid_0's l2: 2.52273
[2700]	valid_0's l2: 2.34556
[2800]	valid_0's l2: 2.20107
[2900]	valid_0's l2: 2.08871
[3000]	valid_0's l2: 1.98411
[3100]	valid_0's l2: 1.90127
[3200]	valid_0's l2: 1.82554
[3300]	valid_0's l2: 1.

In [0]:
## model parameter
params ={'boosting_type': 'goss',
 'learning_rate': 0.0001,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 20,
            early_stopping_rounds = 50,
            num_boost_round = 20000, 
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 50 rounds.
[5480]	valid_0's l2: 11.4109	valid_0's score: 11.4109
[5500]	valid_0's l2: 42.006	valid_0's score: 42.006
[5520]	valid_0's l2: 92.3407	valid_0's score: 92.3407
Early stopping, best iteration is:
[5470]	valid_0's l2: 3.54301	valid_0's score: 3.46206


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

## model - lgbm goss - 4 (0315-1029) 성능
(3과 거의 비슷해서 안해봐도 될듯) 
하이퍼파라메터 조정  

metric: mape
boosting_type: goss  
'num_leaves': 100


In [0]:
## model parameter
params ={'metric': 'mape',
 'boosting_type': 'goss',
 'learning_rate': 0.0005,
 'num_leaves': 100,
 'random_state': 7,
 'objective': 'regression'}

In [0]:
model1 = lgb.train(params, 
            m1_train_data,
           # feval=mse1,
            valid_sets=m1_validation_data,
            verbose_eval=100,
            num_boost_round = 10000, 
            early_stopping_rounds=30,
            keep_training_booster=True) 

Training until validation scores don't improve for 30 rounds.
[100]	valid_0's mape: 0.213002
[200]	valid_0's mape: 0.203031
[300]	valid_0's mape: 0.193598
[400]	valid_0's mape: 0.184761
[500]	valid_0's mape: 0.17636
[600]	valid_0's mape: 0.168361
[700]	valid_0's mape: 0.160769
[800]	valid_0's mape: 0.153621
[900]	valid_0's mape: 0.146946
[1000]	valid_0's mape: 0.140613
[1100]	valid_0's mape: 0.134582
[1200]	valid_0's mape: 0.128849
[1300]	valid_0's mape: 0.123463
[1400]	valid_0's mape: 0.118364
[1500]	valid_0's mape: 0.1135
[1600]	valid_0's mape: 0.10891
[1700]	valid_0's mape: 0.104593
[1800]	valid_0's mape: 0.100589
[1900]	valid_0's mape: 0.0967461
[2000]	valid_0's mape: 0.0931253
[2100]	valid_0's mape: 0.0774126
[2200]	valid_0's mape: 0.0664663
[2300]	valid_0's mape: 0.0588271
[2400]	valid_0's mape: 0.0532439
[2500]	valid_0's mape: 0.0494256
[2600]	valid_0's mape: 0.0468737
[2700]	valid_0's mape: 0.0447725
[2800]	valid_0's mape: 0.0432375
[2900]	valid_0's mape: 0.0420566
[3000]	valid

In [0]:
## model parameter
params['learning_rate'] =  params['learning_rate']/5

In [0]:
## model parameter
model2 = lgb.train(params, 
            m2_train_data,
            feval = mse1,
            valid_sets = m2_validation_data,
            verbose_eval = 10,
            early_stopping_rounds = 50,
            num_boost_round = 20000, 
            init_model = model1) ## model1을 finetuning 

test_predictions = model2.predict(df_test[x_col], num_iteration=model2.best_iteration)


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:814: UserWarning: The prediction of init_model will be overridden by init_score.
  warnings.warn("The prediction of init_model will be overridden by init_score.")


Training until validation scores don't improve for 50 rounds.
[4820]	valid_0's mape: 0.160725	valid_0's score: 15.9399
[4840]	valid_0's mape: 0.286306	valid_0's score: 50.5004
Early stopping, best iteration is:
[4806]	valid_0's mape: 0.0729171	valid_0's score: 3.46206


In [0]:
now = datetime.now()
now =now.strftime('%m%d_%H%M')

In [0]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':test_predictions.reshape(1,-1)[0]})
submit.to_csv('./results/baseline_result_2_'+now+'.csv', index = False)

# tmp
